<table>
  <tr>
    <td><div align="left"><font size="30">Event camera demo</font></div></td>
    <td><img src="https://github.com/Tobias-Fischer/RVSS2022/blob/main/Robotic_Vision/common/RVSS-logo.png?raw=1" width="400"></td>
  </tr>
</table>

**By Cedric Scheerlinck; edited and adapted for RVSS2022 by Tobias Fischer**

Description: Let's explore event cameras. We will load a small dataset to get familiar with the event data structure, and display some popular visualisations. We will also see what happens if we integrate over the events directly, or with a leaky integrator. Finally, we will explore a promising direction where conventional frames are fused with the event data using a so-called complementarity filter.

This notebook is based on: https://github.com/cedric-scheerlinck/dvs_image_reconstruction  

In [ ]:
# If you are interested to find out more, check out this video!

from IPython.display import YouTubeVideo
YouTubeVideo('bZ0ZKido0Ag')

In [ ]:
# Let's get ready, we need the repo for some helper scripts
if not os.path.exists('RVSS2022'):
  !git clone --depth 1 https://github.com/Tobias-Fischer/RVSS2022.git
else:
  %cd RVSS2022
  !git pull
  %cd ..

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('RVSS2022/Robotic_Vision/common'))

from ipywidgets import interact, fixed, interact_manual, FloatSlider, IntSlider
import math
%matplotlib inline
from matplotlib import rc
rc('animation', html='jshtml')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import ArtistAnimation
from IPython.core.display import display, HTML, Image

# local modules
from util import Timer, Event, normalize_image, animate, load_events, plot_3d, event_slice

In [ ]:
def high_pass_filter(event_data, cutoff_frequency=5):
    print('Reconstructing, please wait...')
    events, height, width = event_data.event_list, event_data.height, event_data.width
    events_per_frame = 2e4
    with Timer('Reconstruction'):
        time_surface = np.zeros((height, width), dtype=np.float32)
        image_state = np.zeros((height, width), dtype=np.float32)
        image_list = []
        for i, e in enumerate(events):
            beta = math.exp(-cutoff_frequency * (e.t - time_surface[e.y, e.x]))
            image_state[e.y, e.x] = beta * image_state[e.y, e.x] + e.p
            time_surface[e.y, e.x] = e.t
            if i % events_per_frame == 0:
                beta = np.exp(-cutoff_frequency * (e.t - time_surface))
                image_state *= beta
                time_surface.fill(e.t)
                image_list.append(np.copy(image_state))
    return animate(image_list, 'High Pass Filter')

def leaky_integrator(event_data, beta=1.0):
    print('Reconstructing, please wait...')
    events, height, width = event_data.event_list, event_data.height, event_data.width
    events_per_frame = 2e4
    with Timer('Reconstruction (simple)'):
        image_state = np.zeros((height, width), dtype=np.float32)
        image_list = []
        for i, e in enumerate(events):
            # Note the beta here. If set to 1, we directly integrate.
            image_state[e.y, e.x] = beta * image_state[e.y, e.x] + e.p
            if i % events_per_frame == 0:
                image_list.append(np.copy(image_state))
    fig_title = 'Direct Integration' if beta == 1 else 'Leaky Integrator'
    return animate(image_list, fig_title)


In [ ]:
# Let's load in the event data

with Timer('Loading'):
    n_events = 5e5
    path_to_events = 'RVSS2022/Robotic_Vision/event_camera_data/boxes_6dof/events.zip'
    event_data = load_events(path_to_events, n_events)        

In [ ]:
# Let's have a look how events are specified

event_data.event_list[:10]

## Event point cloud

<span style="color:red">Red = ON event (+1),</span> <span style="color:blue">blue = OFF event (-1).</span>  
Click and drag the 3D plot to rotate it.

In [ ]:
with Timer('3D plot'):
    plot_3d(event_data, n_events=10e3)

## Event slice

Use the sliders to visualize a slice of events within a time window.

In [ ]:
interact(event_slice, event_data=fixed(event_data),
         start=FloatSlider(min=0, max=1, step=0.01,continuous_update=False),
         duration_ms=IntSlider(value=50, min=0, max=500, step=1, continuous_update=False));

## Image reconstruction
Run the cell, then click ```Run Interact``` and wait for the animation to appear to see the result.  
Use the slider then click ```Run Interact``` again to try different settings.

### Leaky integrator
Equation: $L^{k+1}(x, y) = \beta L^k(x, y) + p,\ \beta \in [0, 1],\ p \in \{-1, 1\}$, where $L$ is the log image intensity and $p$ is the event polarity.

#### Set ```beta = 1``` for direct integration. Try lowering beta to see what happens.

In [ ]:
interact_manual(leaky_integrator, event_data=fixed(event_data), beta=(0, 1, 0.01));

### High pass filter
Below is the high pass filter as described in: https://cedric-scheerlinck.github.io/files/2018_scheerlinck_continuous-time_intensity_estimation.pdf  
Equation: $L^{k+1}(x, y) = \exp(-\alpha \Delta t) L^k(x, y) + p$, where $\alpha$ is the cutoff frequency and $\Delta t$ is the time since the last event at the same pixel.

In [ ]:
interact_manual(high_pass_filter, event_data=fixed(event_data), cutoff_frequency=(0, 20, 0.01));

### Complementary filter
Below is the complementary filter as described in: https://cedric-scheerlinck.github.io/files/2018_scheerlinck_continuous-time_intensity_estimation.pdf 

The basic idea is to fuse event data with the conventional frame output that (some) event cameras provide. Remember that just with the event data we don't know the initial state of the image, the conventional frame helps us out.

Equation: $L^{k+1}(x, y) = \exp(-\alpha \Delta t) L^k(x, y) + (1 - \exp(-\alpha \Delta t)) L_F(x, y) + p$, where $\alpha$ is the cutoff frequency, $\Delta t$ is the time since the last event at the same pixel and $L_F$ is the log intensity of the image frame from a frame-based camera, e.g., DAVIS.

In [ ]:
# Let's load the conventional frames
event_data.add_frame_data('RVSS2022/Robotic_Vision/event_camera_data/boxes_6dof')

In [ ]:
def complementary_filter(event_data, cutoff_frequency=5.0):
    print('Reconstructing, please wait...')
    events, height, width = event_data.event_list, event_data.height, event_data.width
    frames, frame_timestamps = event_data.frames, event_data.frame_timestamps
    events_per_frame = 2e4
    with Timer('Reconstruction'):
        time_surface = np.zeros((height, width), dtype=np.float32)
        image_state = np.zeros((height, width), dtype=np.float32)
        image_list = []
        frame_idx = 0
        max_frame_idx = len(frames) - 1
        log_frame = np.log(frames[0] + 1)
        for i, e in enumerate(events):
            if frame_idx < max_frame_idx:
                if e.t >= frame_timestamps[frame_idx + 1]:
                    log_frame = np.log(frames[frame_idx + 1] + 1)
                    frame_idx += 1
            beta = math.exp(-cutoff_frequency * (e.t - time_surface[e.y, e.x]))
            image_state[e.y, e.x] = beta * image_state[e.y, e.x] \
                                    + (1 - beta) * log_frame[e.y, e.x] + 0.1 * e.p
            time_surface[e.y, e.x] = e.t
            if i % events_per_frame == 0:
                beta = np.exp(-cutoff_frequency * (e.t - time_surface))
                image_state = beta * image_state + (1 - beta) * log_frame
                time_surface.fill(e.t)
                image_list.append(np.copy(image_state))
    return animate(image_list, 'Complementary Filter')

In [ ]:
interact_manual(complementary_filter, event_data=fixed(event_data), cutoff_frequency=(0, 20, 0.01));